# $\large{\color{CadetBlue}{\textbf{Escolhendo uma Cerveja Artesanal 🍺}}}$

### Um datasey de mais de 73k linhas, foi realizado uma análise exploratória, a fim de descobrir qual era o estilo de bebida de acordo com as variáveis preditoras, no final, esta quantidade de linhas foi reduzida para 35k e a partir deste momento, foi testado 4 modelos de Classificação:

1. Regressão Logística
2. Naive Bayes
3. KNN
4. Decision Tree


<a id = "table-of-content"></a>
# $\large{\color{CadetBlue}{\textbf{Sumário 📑}}}$ 

- **[1. Carregando as Bibliotecas 📚](#lib)**
- **[2. Lendo os dados 👀](#ler)**
- **[3. Análise Exploratória dos Dados 🔎](#análise)**
    - [3.1. Proporção das colunas e linhas 📋](#análise1)
    - [3.2. Observando o tipo das Variáveis 🔢](#análise2)
    - [3.3. Verificando se há Valores Nulos ❌](#análise3)
    - [3.4. Excluindo Colunas Desnecessárias 🧹](#análise3.1)
    - [3.5. Proporção dos valores da Variável Alvo 🎯](#análise5)
    - [3.6. Substituindo dados missing por valores ✨](#análise6)
    - [3.7. Separando Variáveis Preditoras e Variável Alvo 🎯](#análise4)
- **[4. Criação do Modelo 🧠](#modelo)**
    - [4.1. Rodando 4 Modelos de Classificação 💻](#kfold)
    - [4.2. KNN 🏙️](#knn)
        - [4.2.1. Aplicação do GridSearchCV 🤖](#grid)
        - [4.3.2. Treinando o Modelo 🏋️](#fit1) 
        - [4.3.3. Visualizando os resultados 🖨️](#print1) 
    - [4.3. Decision Tree 🌳 ](#tree) 
        - [4.3.1. Aplicação do GridSearchCV 🤖](#grid2)
        - [4.3.2. Treinando o Modelo 🏋️](#fit) 
        - [4.3.3. Visualizando os resultados 🖨️](#print)     

## $\large{\color{RoyalBlue}{1.}}$ $\large{\color{CadetBlue}{\textbf{Carregando as Bibliotecas 📚}}}$ <a id = "lib"></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import graphviz
import subprocess
import os
from IPython.display import Image
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

## $\large{\color{RoyalBlue}{2.}}$ $\large{\color{CadetBlue}{\textbf{Lendo os dados 👀}}}$ <a id = "ler"></a>

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pd.set_option('display.max.columns',23)
df = pd.read_csv("/kaggle/input/beer-recipes/recipeData.csv", encoding='latin1')
df.head()

In [ ]:
df.tail()

## $\large{\color{RoyalBlue}{3.}}$ $\large{\color{CadetBlue}{\textbf{Análise Exploratória dos Dados 🔎}}}$ <a id = "análise"></a>

##### $\large{\color{RoyalBlue}{3.1.}}$ $\large{\color{CadetBlue}{\textbf{Proporção das colunas e linhas 📋}}}$ <a id = "análise1"></a>

In [ ]:
df.shape

##### $\large{\color{RoyalBlue}{3.2.}}$ $\large{\color{CadetBlue}{\textbf{Observando o tipo das Variáveis 🔢}}}$ <a id = "análise2"></a>

In [ ]:
df.dtypes

##### $\large{\color{RoyalBlue}{3.3.}}$ $\large{\color{CadetBlue}{\textbf{Verificando se há Valores Nulos ❌}}}$ <a id = "análise3"></a>

In [ ]:
faltantes = (df.isnull().sum()/len(df['BeerID']))*100
print(faltantes)

#### Algumas variáveis praticamente não possuem dados, como é o caso da _'PrimingAmount'_, com 93,53% de dados nulos. Para este modelo eu decidi excluir colunas com mais de 70% dos dados nulos.

In [ ]:
df.drop('PrimingMethod',axis = 1, inplace = True)
df.drop('PrimingAmount',axis = 1, inplace = True)


##### $\large{\color{RoyalBlue}{3.4.}}$ $\large{\color{CadetBlue}{\textbf{Excluindo Colunas Desnecessárias 🧹}}}$ <a id = "análise5=3.1"></a> 

#### Aproveitando que estamos excluindo colunas, outras colunas que podem ser excluídas são as desnecessárias, logo de cara, podemos excluir a coluna _'Name'_ e _'Style'_, porque para amnas as colunas possuímos uma outra coluna com um ID respectivo. Outra coluna é a URL, com o link para a cerveja, esta informação não ajuda em nada no modelo, assim como o ID do usuário que participou da pesquisa.

#### Vou aplicar a técnica de Feature Selection para avaliar quais são as variáveis que mais possuem influência com a nossa variável alvo, o StyleID.

In [ ]:
df.drop('Name',axis = 1, inplace = True)
df.drop('Style',axis = 1, inplace = True)
df.drop('URL',axis = 1, inplace = True)
df.drop('UserId',axis = 1, inplace = True)
df.drop('BeerID',axis = 1, inplace = True)


In [ ]:
df.dtypes

#### Sobrou duas variáveis do tipo object, vamos analisar o que  faremos com elas:

In [ ]:
contagem = df['SugarScale'].value_counts()
proporcao = (contagem / len(df))*100

print('Quantidade: ', contagem)
print()
print('Porcentagem: ',proporcao)

#### 97% dos dados são a mesma resposta, além de termos apenas 2 opções de resposta, então vamos transformar os dados em 0 e 1

In [ ]:
df['SugarScale'] = df['SugarScale'].replace('Specific Gravity',0)
df['SugarScale'] = df['SugarScale'].replace('Plato',1)

In [ ]:
contagem = df['BrewMethod'].value_counts()
proporcao = (contagem / len(df))*100

print('Quantidade: ', contagem)
print()
print('Porcentagem: ',proporcao)

#### Já os dados da variável _'BrewMethod'_ possuem 4 tipos de registro, então vou aplicar o método de **One-Hot Code**

In [ ]:
brewMethod_encode = pd.get_dummies(df['BrewMethod'])
df.drop('BrewMethod',axis=1,inplace=True)
concatenado = pd.concat([df,brewMethod_encode],axis=1)
concatenado.head(3)

##### $\large{\color{RoyalBlue}{3.5.}}$ $\large{\color{CadetBlue}{\textbf{Proporção dos valores da Variável Alvo 🎯}}}$ <a id = "análise5"></a>

In [ ]:
contagem = concatenado['StyleID'].value_counts()
proporcao = (contagem / len(df))*100

print('Quantidade: ', contagem.to_string())
print()
print('Porcentagem: ',proporcao)

#### Temos 176 estilos de cervejas artesanais, porém, a quantidade de amostra para cada estilo é bem discrepante, tendo a maior amostra com mais de 11 mil registros e a menor com apenas 2. É preciso olhar a proporção e definir um threshold, ou seja, uma linha de corte. Neste exercício vou considerar apenas valores maiores que mil amostra por tipo de cerveja (_'StyleID'_).

In [ ]:
plt.figure(figsize=(12, 8))
plt.hist(concatenado['StyleID'], bins=176)
plt.axhline(y=1000, color='r', linestyle='--', label='Threshold')
plt.title('Quantidade de linhas')
plt.xlabel('StyleID')
plt.ylabel('Contagem')
plt.show()

In [ ]:
filtro = concatenado.loc[concatenado['StyleID'].isin([7,10,134,9,4,30,86,12,92,6,175,39])]
filtro.shape

#### De 70 mil linhas, nosso novo dataset tem 35 mil, mas agora está mais consistente.

##### $\large{\color{RoyalBlue}{3.6.}}$ $\large{\color{CadetBlue}{\textbf{Substituindo dados missing por valores ✨}}}$ <a id = "análise6"></a>

In [ ]:
faltantes = (filtro.isnull().sum()/len(filtro['StyleID']))*100
print(faltantes)

#### Ainda temos 4 variáveis com dados missing, por ser variáveis numéricas, podemos substituir os valores nulos pela média, mediana e afins, encontrando um dado que comporte de maneira razoável naquela distribuição numérica.

In [ ]:
filtro.boxplot(column=['BoilGravity','MashThickness','PitchRate','PrimaryTemp'])
plt.show()

#### Tem muito outlier, mas eu ainda não sei tratar eles de forma apropriada, pelo menos neste primeiro momento que lanço este notebook, vou ignorar esses outliers.

In [ ]:
filtro.hist(column=['BoilGravity','MashThickness','PitchRate','PrimaryTemp'],bins=20)
plt.show()

#### Olhando a distibuição desses dados, o _'PitchRate'_ segue uma distribuição com uma linha de tendência com queda, isso quer dizer que teremos pontos maiores no começo e menores no final, então a média desses valores pode pegar um ponto que esteja razoável para ambas extremidades. Já os outros gráficos não possuem este padrão, onde a mediana possa representar melhor um valor neutro, de baixa influência para o modelo.

In [ ]:
filtro['PitchRate'].fillna(filtro['PitchRate'].mean(),inplace=True)
filtro.fillna(filtro.median(),inplace=True)

In [ ]:
faltantes = (filtro.isnull().sum()/len(filtro['StyleID']))*100
print(faltantes)

##### $\large{\color{RoyalBlue}{3.7.}}$ $\large{\color{CadetBlue}{\textbf{Separando Variáveis Preditoras e Variável Alvo 🎯}}}$ <a id = "análise4"></a>

In [ ]:
y = filtro['StyleID']
x = filtro.drop('StyleID', axis=1)

## $\large{\color{RoyalBlue}{4.}}$ $\large{\color{CadetBlue}{\textbf{Criação do Modelo 🧠}}}$ <a id = "modelo"></a>

##### $\large{\color{RoyalBlue}{4.1.}}$ $\large{\color{CadetBlue}{\textbf{Rodando 4 Modelos de Classificação 💻}}}$ <a id = "kfold"></a>

In [ ]:
def modelosclassificacao(a,b):

    skfold= StratifiedKFold(n_splits=3)
    
    x = a
    y = b
    
    # Normalizando as variáveis preditoras para o KNN
    normalizador = MinMaxScaler(feature_range=(0,1))
    x_norm = normalizador.fit_transform(x)
    
    logist = LogisticRegression()
    naive = GaussianNB()
    decision_tree = DecisionTreeClassifier()
    knn = KNeighborsClassifier()
    
    resultado_logist = cross_val_score(logist,x,y,cv=skfold)
    resultado_naive = cross_val_score(naive,x,y,cv=skfold)
    resultado_decision_tree = cross_val_score(decision_tree,x,y,cv=skfold)
    resultado_knn = cross_val_score(knn,x_norm,y,cv=skfold)
    
    dic_classmodels = {'Logística':resultado_logist.mean(),'naive':resultado_naive.mean(), 'Decision Tree':resultado_decision_tree.mean(), 'KNN':resultado_knn.mean()}
    melhor_modelo = max(dic_classmodels, key=dic_classmodels.get)
    
    print('Regressão Logística: ',resultado_logist.mean(),'Naive Bayes: ',resultado_naive.mean(),'Decision Tree: ',resultado_decision_tree.mean(),'KNN: ',resultado_knn.mean())
    print('Melhor modelo foi: ',melhor_modelo,'com o valor: ',dic_classmodels[melhor_modelo])
                                       
   


In [ ]:
modelosclassificacao(x,y)

#### Os melhores resultados foram o KNN e o Decision Tree, ambos com 45% de acurácia, como são 2 modelos que estão com valores próximos, em um teste realizado com hiperparametros padrões, ainda é possível buscar um resultado melhor testando diversos valores de hiperparametros com o **GridSearchCV**.

##### $\large{\color{RoyalBlue}{4.2.}}$ $\large{\color{CadetBlue}{\textbf{KNN 🏙️}}}$ <a id = "knn"></a>

##### $\large{\color{RoyalBlue}{4.2.1.}}$ $\large{\color{CadetBlue}{\textbf{Aplicação do GridSearchCV 🤖}}}$ <a id = "grid"></a>

In [ ]:
# Normalizando as variáveis preditoras 
normalizador = MinMaxScaler(feature_range=(0,1))
x_norm = normalizador.fit_transform(x)

#Definindo os valores que serão testados no KNN:
valores_K  = np.array([3,5,7])
calculo_distancia = ['minkowski','chebyshev']
valores_p = np.array([1,2,3])
valores_grid = {'n_neighbors':valores_K,'metric':calculo_distancia,'p':valores_p}

##### $\large{\color{RoyalBlue}{4.2.2.}}$ $\large{\color{CadetBlue}{\textbf{Treinando o Modelo 🏋️}}}$ <a id = "fit1"></a>

In [ ]:
# Criação do modelo:
modelo = KNeighborsClassifier()

# Criando os grids:
gridKNN = GridSearchCV(estimator = modelo,param_grid = valores_grid, cv = 3,n_jobs= -1)
gridKNN.fit(x_norm,y)

##### $\large{\color{RoyalBlue}{4.2.3.}}$ $\large{\color{CadetBlue}{\textbf{Visualizando os resultados 🖨️}}}$ <a id = "print1"></a>

In [ ]:
# Imprimindo os melhores parâmetros:
print('Melhor acurácia: ', gridKNN.best_score_)
print('Melhor K: ', gridKNN.best_estimator_.n_neighbors)
print('Método distância: ', gridKNN.best_estimator_.metric)
print('Melhor valor p: ', gridKNN.best_estimator_.p)

##### $\large{\color{RoyalBlue}{4.3.}}$ $\large{\color{CadetBlue}{\textbf{Decision Tree 🌳}}}$ <a id = "tree"></a>

##### $\large{\color{RoyalBlue}{4.3.1.}}$ $\large{\color{CadetBlue}{\textbf{Aplicação do GridSearchCV 🤖}}}$ <a id = "grid2"></a>

In [ ]:
# Definindo os valores que serão testados em DecisionTree
minimos_split = np.array([2,3,4,5,6,7])
maximo_nivel = np.array([3,4,5,6,7])
algoritmo = ['gini','entropy', 'log_loss']
valores_grid = {'min_samples_split':minimos_split,'max_depth':maximo_nivel,'criterion':algoritmo}

##### $\large{\color{RoyalBlue}{4.3.2.}}$ $\large{\color{CadetBlue}{\textbf{Treinando o Modelo 🏋️}}}$ <a id = "fit"></a>

In [ ]:
modelo = DecisionTreeClassifier()

grid = GridSearchCV(estimator = modelo, param_grid = valores_grid)
grid.fit(x,y)

##### $\large{\color{RoyalBlue}{4.3.3.}}$ $\large{\color{CadetBlue}{\textbf{Visualizando os resultados 🖨️}}}$ <a id = "print"></a>

In [ ]:
print('Mínimo split: ', grid.best_estimator_.min_samples_split)
print('Máxima profundidade: ', grid.best_estimator_.max_depth)
print('Algoritmo escolhido: ', grid.best_estimator_.criterion)
print('Acurácia: ', grid.best_score_)

#### De 47% foi possível aumentar a acurácia para 58%, mais de 10%, um aumento significativo através de parâmetros melhores ajustados.
#### Diferente do KNN que teve um aumento de apenas 2%, mesmo tendo custado muito mais do processamento do que o Decision Tree.